In [10]:
## asymmetric demand and inventory level
## New experiments for Asymmetric demand and modified balancing policy

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import random
import math
import csv
from collections import Counter
import itertools
import copy
import os
import glob

def simulate_ER(n,prob, opq_prob, opq_Set,S, iteration):
    # directly balance inventory
    # S is a list of size n
    
    R_list = []
    leftover_list = []
    end_list = []
    num_choice = n + len(opq_Set)
    population = list(range(num_choice))
    weight = prob+opq_prob
    for t in range(iteration):
        inv = [S[i] for i in range(n)]
        flag = 1
        while flag ==1:
            choice = random.choices(population, weights=weight, k=1)[0]
            if choice<n:
                inv[choice] = inv[choice]-1
            else:
                choice = opq_Set[choice-n]
                index_max = np.argmax([inv[index] for index in choice])
                inv[choice[index_max]] = inv[choice[index_max]] -1
            if min(inv)==0:
                R_list.append(sum(S)-sum(inv))
                end_list.append(inv.index(min(inv)))
                leftover_list.append(inv.index(max(inv)))
                flag = 0
    return(np.average(R_list), np.average([i**2 for i in R_list]), end_list, leftover_list)



def simulate_ER_greedy2(n,prob, opq_prob, opq_Set,S, iteration):
    # S is a list of size n
    
    R_list = []
    leftover_list = []
    end_list = []
    num_choice = n + len(opq_Set)
    population = list(range(num_choice))
    weight = prob+opq_prob
    den = [(prob[i] + sum([opq_prob[k] if i in opq_Set[k] else 0 for k in range(len(opq_Set)) ])) for i in range(n)]

    for t in range(iteration):
        inv = [S[i] for i in range(n)]
        flag = 1
        while flag ==1:
            choice = random.choices(population, weights=weight, k=1)[0]
            if choice<n:
                inv[choice] = inv[choice]-1
            else:
                choice = opq_Set[choice-n]
                greedy = []
                greedy = [ float(inv[i])/den[i] for i in range(n)]
                index_max = np.argmax([greedy[index] for index in choice])
                inv[choice[index_max]] = inv[choice[index_max]] -1
            if min(inv)==0:
                R_list.append(sum(S)-sum(inv))
                end_list.append(inv.index(min(inv)))
                leftover_list.append(inv.index(max(inv)))
                flag = 0
    return(np.average(R_list), np.average([i**2 for i in R_list]), end_list, leftover_list)


def cost(n,h,K,lamb, S, ER,ER2):
    initial_inv = sum(S)
    holding = (2*initial_inv+1)*ER-ER2
    holding = float(h*holding)/(2*lamb*ER)
    ordering = float(K)/ER
    return(holding+ordering, holding, ordering)


"""
n opq
q_i,F notation
"""

n=3
K=1500
h=1
lamb=1

# four inventory levels
S1 = [20, 20 , 20]
S2 = [16, 20, 24]
S3 = [12, 20, 28]
S4 = [8, 20, 32]

# three scenarios
q1 =  [[0.4], [0.4], [0.4]]
q2 = [[0.2], [0.4], [0.6]]
q3 = [[0.6], [0.4], [0.2]]

####################

S = S2
q= q2 # change S and q for other inventory levels and scenarios

prob = [S[i]/sum(S) for i in range(len(S))] # p_i^0
opq_Set = [[i for i in range(n)]]
prob_new = [prob[i]*(1- sum(q[i])) for i in range(n)]
opq_prob = [sum([prob[i]*q[i][j] for i in range(n)]) for j in range(1)]
print("total q = " + str(sum(opq_prob)))

# traditional selling, no opaque
opq_Set = [[i for i in range(n)]]
opq = [0]
[ER_0, ER2, end, left] = simulate_ER(n,prob, opq, opq_Set,S, 20000)
cost0 = cost(n,h,K,lamb, S, ER_0,ER2)

[ER_q, ER2, end, left] = simulate_ER_greedy2(n,prob_new, opq_prob, opq_Set,S, 20000)
cost1 = cost(n,h,K,lamb, S, ER_q,ER2)
print("Relative Savings: "+str(100*float(cost0[0]-cost1[0])/cost0[0]))


ER_FullFlexible = sum(S)-n+1
cost1 = cost(n,h,K,lamb, S, ER_FullFlexible, ER_FullFlexible*ER_FullFlexible)
print("Full Flexibility Relative Saving: "+str( 100*float(cost0[0]-cost1[0])/cost0[0]) )

total q = 0.42666666666666664
Relative Savings: 10.649291598023344
Full Flexibility Relative Saving: 13.761601666349188


In [15]:
"""
2opq scenarios
"""

# four inventory levels
S1 = [20, 20 , 20]
S2 = [16, 20, 24]
S3 = [12, 20, 28]
S4 = [8, 20, 32]

# three scenarios
Q1 =  [[0.1, 0, 0.1] , [0.1, 0.1, 0], [0,0.1, 0.1]] # Q1[0] = q_1 F for all F
Q2 = [[0.05, 0, 0.05] , [0.1, 0.1, 0], [0,0.15, 0.15]]
Q3 = [[0.15, 0, 0.15] , [0.1, 0.1, 0], [0,0.05, 0.05]]

# q = 0.2 version
q1 = [[i[j] for j in range(len(i))] for i in Q1] # for q = 0.4, use i[j]*2
q2 = [[i[j] for j in range(len(i))] for i in Q2]
q3 = [[i[j] for j in range(len(i))] for i in Q3]


# change S and q for different inventory levels and scenarios
S = S3
q = q3
prob = [S[i]/sum(S) for i in range(len(S))] # p_i^0
prob_new = [prob[i]*(1-  sum(q[i])) for i in range(n)]
opq_prob = [sum([prob[i]*q[i][j] for i in range(n)]) for j in range(n)]
print("total q = "+ str(sum(opq_prob)))

n=3
K=1500
h=1
lamb=1

# traditional selling, no opaque
opq_Set = [[i for i in range(n)]]
opq = [0]
[ER_0, ER2, end, left] = simulate_ER(n,prob, opq, opq_Set,S, 20000)
cost0 = cost(n,h,K,lamb, S, ER_0,ER2)
opq_Set = [[0,1], [1,2], [0,2]]
[ER_q, ER2, end, left] = simulate_ER_greedy2(n,prob_new, opq_prob, opq_Set,S, 20000)
cost1 = cost(n,h,K,lamb, S, ER_q,ER2)
print("Relative Savings: "+str(100*float(cost0[0]-cost1[0])/cost0[0]))

total q = 0.17333333333333334
Relative Savings: 6.708748896389195
